In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sys
import h3
import pandas as pd
from libpysal.weights import W
import pymc as pm

N = 7 # 13
RES =res = 9 # need to test the 8 later
N_CAT = 27
import os
# prefix = "built_env"
# prefixfull = "_built_environment"
# merge both prefix
prefix_ls = ["_built_environment", ""]
ROOT = "/lustre1/g/geog_pyloo/05_timemachine/"
CURATED_FOLDER = f"{ROOT}/_curated/"
EXPORT_FOLDER = f"{ROOT}/_curated/c_analysis"

In [ ]:
prefix = "_built_environment"
continent_sel = ['Asia', 'Europe', 'North America',  'South America']

hex_path = os.path.join(EXPORT_FOLDER,f"c_hex{prefix}_full_cluster={N}_ncat={N_CAT}.csv")
hex_df = pd.read_csv(hex_path)
hex_df = hex_df[hex_df['include']!=0]
hex_df.columns = [x.lower() for x in hex_df.columns]

In [23]:
continent = "North America"
df = hex_df[hex_df['continent'] == continent].copy()

# Make mapping from h3_id to row index
h3_index = {h: i for i, h in enumerate(df['hex_id'])}

In [24]:


# Suppose df is your dataframe
adjacency = {
    h: set(h3.grid_ring(h, 1)) - {h}  # immediate neighbors 4.x API
    for h in df['hex_id']
}
print(len(adjacency.keys()))

# Create weight dictionary with integer indices
neighbors = {
    h3_index[h]: [h3_index[n] for n in adjacency[h] if n in h3_index]
    for h in df['hex_id']
}

w = W(neighbors)

48117


/tmp/ipykernel_3956484/1221436251.py:14: UserWarning: The weights matrix is not fully connected: 
 There are 542 disconnected components.
 There are 301 islands with ids: 17, 51, 52, 146, 149, 150, 187, 382, 399, 439, 441, 507, 537, 551, 562, 603, 651, 896, 897, 905, 914, 915, 920, 921, 922, 924, 925, 926, 933, 938, 940, 941, 942, 944, 949, 951, 954, 955, 960, 965, 969, 970, 974, 977, 980, 984, 992, 994, 1001, 1002, 1003, 1008, 1010, 1014, 1021, 1022, 1030, 1031, 1032, 1034, 1038, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1056, 1057, 1058, 1059, 1069, 1075, 1079, 1081, 1082, 1088, 1089, 1090, 1091, 1094, 1106, 1110, 1112, 1116, 1117, 1122, 1123, 1127, 1132, 1133, 1136, 1137, 1154, 1155, 1156, 1160, 1161, 1162, 1163, 1164, 1171, 1173, 1176, 1177, 1178, 1182, 1191, 1195, 1196, 1197, 1198, 1199, 1203, 1204, 1205, 1206, 1208, 1210, 1213, 1216, 1221, 1222, 1223, 1225, 1230, 1234, 1241, 1252, 1255, 1258, 1259, 1262, 1265, 1266, 1267, 1268, 1272, 1274, 1281, 1282

In [46]:
neighbors

{0: [37, 34, 3],
 1: [5, 3, 4, 2],
 2: [5, 37, 3, 1, 35],
 3: [37, 1, 2, 0],
 4: [5, 85, 84, 1],
 5: [85, 4, 1, 2],
 6: [8, 7, 9],
 7: [6, 32, 15, 13],
 8: [26, 27, 6, 9],
 9: [6, 16, 8],
 10: [14, 12, 11, 15, 13],
 11: [182, 14, 10],
 12: [217, 19, 10, 15],
 13: [14, 10, 32, 7, 15],
 14: [10, 11, 13],
 15: [10, 12, 7, 13],
 16: [9],
 17: [],
 18: [22, 19, 21, 23, 20],
 19: [217, 21, 12, 18],
 20: [21, 23, 18],
 21: [19, 20, 18, 221],
 22: [23, 18],
 23: [22, 20, 18],
 24: [26, 27, 28, 29, 25, 30],
 25: [27, 33, 29, 426, 24],
 26: [36, 27, 8, 24, 30],
 27: [26, 8, 25, 24],
 28: [90, 29, 80, 24, 30],
 29: [28, 80, 25, 426, 24],
 30: [26, 36, 28, 90, 91, 24],
 31: [33, 32],
 32: [31, 7, 13],
 33: [31, 25, 426],
 34: [37, 0],
 35: [36, 37, 91, 2],
 36: [26, 91, 35, 30],
 37: [34, 3, 0, 2, 35],
 38: [39],
 39: [43, 38, 42],
 40: [225],
 41: [44, 46, 43, 42, 45],
 42: [46, 43, 39, 41],
 43: [39, 42, 41],
 44: [46, 113, 120, 45, 41, 114],
 45: [44, 41, 114],
 46: [44, 122, 120, 42, 41],
 47:

In [25]:
# adjust the classes
cluster_7_dict = {
    0:"Open Arterials",
    1:"High-rise Community",
    2:"Scenic View",
    3:"Suburban Fringe",
    4:"Dense Road",
    5:"Low-rise Community",
    6:"Scenic View"
}
cluster_7_dict_lower = {k:v.lower().replace(" ", "_").replace("-", "_") for k,v in cluster_7_dict.items()}
cluster_7_dict_lower

{0: 'open_arterials',
 1: 'high_rise_community',
 2: 'scenic_view',
 3: 'suburban_fringe',
 4: 'dense_road',
 5: 'low_rise_community',
 6: 'scenic_view'}

In [ ]:
# scale the variables
df['cluster_7'] = df['cluster_7'].astype(int)
df['cluster_7des'] = df['cluster_7'].map(cluster_7_dict_lower)
exposure_cols = """obj_person obj_bicycle obj_motorcycle obj_car obj_bus obj_truck""".split()
seg_cols = """sky road sidewalk building skyscraper tree""".split()
logexposure_cols = [f"log{x}" for x in exposure_cols]
for c in exposure_cols+["urban_pop", "gdp_per_cap_country_2022"]+seg_cols:
    df[f"log{c}"] = np.log(df[c].fillna(0)+1)
cat_cols = ['country_clean', 'cluster_7des']
drop_cols = ["cluster_7des_scenic_view", "country_clean_United States of America"]
num_cols = ["logurban_pop", "loggdp_per_cap_country_2022"]+logexposure_cols + ["log"+x for x in seg_cols]
target = 'waze_major_count'

# encode the categorical variables
df = pd.get_dummies(df, columns=cat_cols).drop(columns=drop_cols)
# standardize the numerical variables
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [42]:
df[target] = df[target].fillna(0)

In [29]:
target = 'waze_major_count'

In [28]:
allvariables = ["logurban_pop", "loggdp_per_cap_country_2022"]+logexposure_cols+list(df.columns[df.columns.str.startswith("country_clean")])\
    +list(df.columns[df.columns.str.startswith("cluster_7des")])
allvariables = [x for x in allvariables if x not in drop_cols]
print(len(allvariables))

14


In [34]:
import scipy.sparse as sp

# Assuming 'w' is your libpysal.weights.W object
sparse_matrix = w.sparse

In [45]:
# check nan for all variables
df[allvariables].isnull().sum()

logurban_pop                        0
loggdp_per_cap_country_2022         0
logobj_person                       0
logobj_bicycle                      0
logobj_motorcycle                   0
logobj_car                          0
logobj_bus                          0
logobj_truck                        0
country_clean_Canada                0
cluster_7des_dense_road             0
cluster_7des_high_rise_community    0
cluster_7des_low_rise_community     0
cluster_7des_open_arterials         0
cluster_7des_suburban_fringe        0
dtype: int64

In [44]:
# Model
import pytensor
import pytensor.tensor as pt
W = pytensor.shared(sparse_matrix)
n = df.shape[0]
mu_vec = np.zeros(n)
fixed_alpha = 0.8
with pm.Model() as model:
    beta = pm.Normal('beta', mu=0, sigma=10, shape=(X.shape[1],))
    tau = pm.Gamma('tau', alpha=1, beta=1)
    # Use fixed_alpha instead of a random variable
    phi = pm.CAR('phi', mu=mu_vec, W=W, alpha=fixed_alpha, tau=tau, shape=n)
    
    eta = pm.math.dot(X, beta) + phi
    mu = pm.math.exp(eta)
    
    # Define dispersion for the NB likelihood
    alpha_nb = pm.HalfCauchy('alpha_nb', beta=2)
    y_obs = pm.NegativeBinomial('y_obs', mu=mu, alpha=alpha_nb, observed=y)
    
    trace = pm.sample(1000, tune=1000, target_accept=0.95)

Initializing NUTS using jitter+adapt_diag...


LinAlgError: 47817-th leading minor not positive definite
Apply node that caused the error: car_rv{"(m),(m,m),(),(),()->(m)"}(RNG(<Generator(PCG64) at 0x14E09288F5A0>), [], [0. 0. 0. ... 0. 0. 0.], SparseTensorSharedVariable{csr,float64}, 0.8, 1.0, And.0)
Toposort index: 20
Inputs types: [RandomGeneratorType, TensorType(int64, shape=(0,)), TensorType(float64, shape=(48117,)), Sparse(float64, (None, None), csr), TensorType(float64, shape=()), TensorType(float64, shape=()), TensorType(bool, shape=())]
Inputs shapes: ['No shapes', (0,), (48117,), (48117, 48117), (), (), ()]
Inputs strides: ['No strides', (0,), (8,), 'No strides', (), (), ()]
Inputs values: [Generator(PCG64) at 0x14E09288F5A0, array([], dtype=int64), 'not shown', 'not shown', array(0.8), array(1.), array(True)]
Outputs clients: [[output[4](car_rv{"(m),(m,m),(),(),()->(m)"}.0)], []]

Backtrace when the node is created (use PyTensor flag traceback__limit=N to make it longer):
  File "/tmp/ipykernel_3956484/3279930900.py", li

In [32]:
# check version of pymc
print(pm.__version__)

5.21.1
